In [20]:
import torch
import pathpyG as pp
from collections import defaultdict, Counter

from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index
from torch_geometric.data import Data


from pathpyG import Graph, TemporalGraph, MultiOrderModel

pp.config['torch']['device'] = 'cpu'

In [4]:
tedges = [('a', 'b', 1),('a', 'b', 2), ('b', 'a', 3), ('b', 'c', 3), ('d', 'c', 4), ('a', 'b', 4), ('c', 'b', 4),
              ('c', 'd', 5), ('b', 'e', 5), ('c', 'b', 6)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.mapping)
print(t.N)
print(t.M)

a -> 0
b -> 1
c -> 2
d -> 3
e -> 4

5
10


In [16]:
def time_respecting_paths(g: TemporalGraph, delta: float) -> defaultdict:
    """
    Calculate all longest time-respecting paths in a temporal graph.
    """
    paths_of_length = {}
    in_degree = degree(g.data.edge_index[1], num_nodes=g.N)

    # first-order edge index
    edge_index, timestamps = sort_edge_index(g.data.edge_index, g.data.t)
    node_sequence = torch.arange(g.data.num_nodes, device=edge_index.device).unsqueeze(1)

    # second-order edge index
    null_model_edge_index = MultiOrderModel.lift_order_edge_index(edge_index, num_nodes=node_sequence.size(0))
    # Update node sequences
    node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
    # Remove non-time-respecting higher-order edges
    time_diff = timestamps[null_model_edge_index[1]] - timestamps[null_model_edge_index[0]]
    non_negative_mask = time_diff > 0
    delta_mask = time_diff <= delta
    time_respecting_mask = non_negative_mask & delta_mask
    edge_index = null_model_edge_index[:, time_respecting_mask]
    
    # calculate degrees
    out_degree = degree(edge_index[0], num_nodes=g.M, dtype=torch.long)
    in_degree = degree(edge_index[1], num_nodes=g.M, dtype=torch.long)
    # identify root nodes with in-degree zero
    roots = torch.where(in_degree == 0)[0]
    leafs = (out_degree == 0)
    paths = node_sequence[roots]
    #paths_of_length[1] = paths[leafs[roots]].tolist()

    paths = paths[~leafs[roots]]
    nodes = roots[~leafs[roots]]

    ptrs = cumsum(out_degree, dim=0)

    # create traversable graph
    event_dag = Graph.from_edge_index(edge_index)

    # count all longest time-respecting paths in the temporal graph
    step = 1
    while nodes.size(0) > 0:
        print("step", step)
        idx_repeat = torch.repeat_interleave(out_degree[nodes])
        next_idx = torch.repeat_interleave(ptrs[nodes], out_degree[nodes])
        idx_correction = torch.arange(next_idx.size(0), device=edge_index.device) - cumsum(out_degree[nodes], dim=0)[idx_repeat]
        next_idx += idx_correction
        next_nodes = edge_index[1][next_idx]
        paths = torch.cat([paths[idx_repeat], node_sequence[next_nodes, 1:]], dim=1)
        # paths_of_length[step] = paths[leafs[next_nodes]].tolist()
        paths = paths[~leafs[next_nodes]]
        nodes = next_nodes[~leafs[next_nodes]]
        # print(edge_index)
        # print([g.mapping.to_id(v[0].item()) for v in node_sequence[nodes]])
        # print(next_idx)
        # print([g.mapping.to_id(v[1].item()) for v in node_sequence[next_nodes]])
        # print(paths)
        step += 1
    return None #paths_of_length


In [21]:
t = pp.TemporalGraph.from_csv('ants_1_1.tedges')
print(t)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	src		<class 'torch.Tensor'> -> torch.Size([1911])
	dst		<class 'torch.Tensor'> -> torch.Size([1911])
	t		<class 'torch.Tensor'> -> torch.Size([1911])



In [22]:
paths = time_respecting_paths(t, delta=30)

step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56


In [15]:
len(paths[30])

383646

In [90]:
for k, v in paths.items():
    print(f"Paths of length {k+1}:")
    for p in v:
        print([t.mapping.to_id(v) for v in p])

Paths of length 2:
['d', 'c', 'b']
['d', 'c', 'd']
Paths of length 3:
['a', 'b', 'c', 'd']
['a', 'b', 'c', 'd']
Paths of length 4:
['a', 'b', 'a', 'b', 'e']
['a', 'b', 'c', 'b', 'e']
['a', 'b', 'a', 'b', 'e']
['a', 'b', 'c', 'b', 'e']


In [86]:
pp.algorithms.centrality.temporal_closeness_centrality(t, delta=5, normalized=False)

Directed graph with 10 nodes and 14 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/3


defaultdict(<function pathpyG.algorithms.centrality.temporal_closeness_centrality.<locals>.<lambda>()>,
            {'b': 0.8333333333333333,
             'd': 0.3333333333333333,
             'e': 0.5,
             'a': 0.0,
             'c': 0.0})

In [7]:
pp.algorithms.centrality.temporal_betweenness_centrality(t, delta=5, normalized=False)

Directed graph with 17 nodes and 7 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/13
Processing root 11/13


defaultdict(<function pathpyG.algorithms.centrality.temporal_betweenness_centrality.<locals>.<lambda>()>,
            {'b': 2.0,
             'c': 2.5,
             'g': 0.5,
             'f': 1.0,
             'a': 1.0,
             'd': 0,
             'e': 0,
             'h': 0,
             'i': 0})

In [5]:
pp.algorithms.temporal_shortest_paths(t, delta=5)

Directed graph with 17 nodes and 7 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/13
Processing root 11/13


(defaultdict(<function pathpyG.algorithms.temporal.temporal_shortest_paths.<locals>.<lambda>()>,
             {'a': defaultdict(set,
                          {'b': {('a', 'b')},
                           'd': {('a', 'b', 'c', 'd')},
                           'e': {('a', 'b', 'c', 'e')},
                           'h': {('a', 'c', 'h'), ('a', 'g', 'h')}}),
              'b': defaultdict(set, {'f': {('b', 'f')}, 'i': {('b', 'i')}}),
              'c': defaultdict(set,
                          {'f': {('c', 'f')},
                           'i': {('c', 'i')},
                           'g': {('c', 'f', 'a', 'g')}}),
              'f': defaultdict(set, {'h': {('f', 'h')}}),
              'h': defaultdict(set, {'f': {('h', 'f')}, 'i': {('h', 'i')}}),
              'i': defaultdict(set, {'b': {('i', 'b')}})}),
 defaultdict(<function pathpyG.algorithms.temporal.temporal_shortest_paths.<locals>.<lambda>()>,
             {'a': defaultdict(<function pathpyG.algorithms.temporal.temporal_shorte

a -> 0
b -> 1
c -> 2
d -> 3

4
7


In [5]:
m = pp.MultiOrderModel.from_temporal_graph(t, max_order=4, delta=2)
print(m.layers[2])
for i, e in enumerate(m.layers[2].edges):
    print(e, m.layers[2].data.edge_weight[i])

Directed graph with 17 nodes and 3 edges

Node attributes
	node_sequence		<class 'torch.Tensor'> -> torch.Size([17, 2])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3])

Graph attributes
	num_nodes		<class 'int'>

(('a', 'c'), ('c', 'h')) tensor(1.)
(('a', 'g'), ('g', 'h')) tensor(1.)
(('c', 'f'), ('f', 'a')) tensor(1.)


In [3]:
# Solution 1 -> slow
def temporal_shortest_paths(g: pp.TemporalGraph, delta, max_k) -> defaultdict:
    """
    Calculates all shortest paths between all pairs of nodes 
    based on a set of empirically observed paths.
    """
    sp = defaultdict(lambda: defaultdict(set))
    sp_lengths = torch.full((g.N, g.N), float('inf'))
    sp_lengths.fill_diagonal_(float(0))
    out_degree=degree(g.data.edge_index[0],num_nodes=g.N)
    in_degree =degree(g.data.edge_index[1],num_nodes=g.N)

    sp_lengths[out_degree==0]=0
    sp_lengths[:,in_degree==0]=0
    print(sp_lengths)

    # first-order edge index
    edge_index, timestamps = sort_edge_index(g.data.edge_index, g.data.t)
    node_sequence = torch.arange(g.data.num_nodes, device=edge_index.device).unsqueeze(1)
    k = 1
    while torch.max(sp_lengths) > k and edge_index.size(1)>0 and k < max_k:

        print(f'k = {k}, edge_index size = {edge_index.size(1)}')
        #print(f'node_sequences = {node_sequences}')
        # check for shorter paths with length k
        src, tgt = edge_index
        for i in range(edge_index.size(1)):
            u = node_sequence[src[i]][0]
            v = node_sequence[tgt[i]][-1]
            if k < sp_lengths[u][v]:
                path = torch.cat([node_sequence[src[i]], v.unsqueeze(dim=0)])
                sp_lengths[u][v] = k
                sp[u][v] = set([g.mapping.to_id(x.item()) for x in path])
            elif k == sp_lengths[u][v]:
                path = torch.cat([node_sequence[src[i]], v.unsqueeze(dim=0)])
                sp[u][v].add(tuple([g.mapping.to_id(x.item()) for x in path]))
                 

        if k==1:
            null_model_edge_index = pp.MultiOrderModel.lift_order_edge_index(edge_index, num_nodes=node_sequence.size(0))
            # Update node sequences
            node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
            # Remove non-time-respecting higher-order edges
            time_diff = timestamps[null_model_edge_index[1]] - timestamps[null_model_edge_index[0]]
            non_negative_mask = time_diff > 0
            delta_mask = time_diff <= delta
            time_respecting_mask = non_negative_mask & delta_mask
            edge_index = null_model_edge_index[:, time_respecting_mask]
        else:
            edge_index, node_sequence, _, _ = pp.MultiOrderModel.iterate_lift_order(edge_index, node_sequence, mapping=g.mapping)
        k += 1
    return sp_lengths, sp


In [5]:
# Solution 2
def routes_from_node(g, v, node_sequence, mapping):
    """
    Constructs all paths from node v to any leaf node

    Parameters
    ----------
    v:
        node from which to start
    node_mapping: dict
        an optional mapping from node to a different set.

    Returns
    -------
    Counter
    """
    # Collect temporary paths, indexed by the target node
    temp_paths = defaultdict(list)
    temp_paths[v] = [[mapping.to_id(node_sequence[v][0].item())]]

    # set of unprocessed nodes
    queue = {v}

    while queue:
        # take one unprocessed node
        x = queue.pop()

        # successors of x expand all temporary
        # paths, currently ending in x
        s = g.get_successors(x)
        for w in s:
            w = w.item()
            for p in temp_paths[x]:
                temp_paths[w].append(p + [mapping.to_id(node_sequence[w][1].item())])
            queue.add(w)
        if s.size(0) > 0:
            del temp_paths[x]
    # flatten dictionary
    return temp_paths    

def temporal_shortest_paths(g: pp.TemporalGraph, delta) -> defaultdict:
    """
    Calculates all shortest paths between all pairs of nodes 
    based on a set of empirically observed paths.
    """
    sp = defaultdict(lambda: defaultdict(set))
    sp_lengths = torch.full((g.N, g.N), float('inf'))
    sp_lengths.fill_diagonal_(float(0))
    out_degree=degree(g.data.edge_index[0],num_nodes=g.N)
    in_degree =degree(g.data.edge_index[1],num_nodes=g.N)

    sp_lengths[out_degree==0]=0
    sp_lengths[:,in_degree==0]=0
    print(sp_lengths)

    # first-order edge index
    edge_index, timestamps = sort_edge_index(g.data.edge_index, g.data.t)
    node_sequence = torch.arange(g.data.num_nodes, device=edge_index.device).unsqueeze(1)
    k = 1

    # second-order edge index
    null_model_edge_index = pp.MultiOrderModel.lift_order_edge_index(edge_index, num_nodes=node_sequence.size(0))
    # Update node sequences
    node_sequence = torch.cat([node_sequence[edge_index[0]], node_sequence[edge_index[1]][:, -1:]], dim=1)
    # Remove non-time-respecting higher-order edges
    time_diff = timestamps[null_model_edge_index[1]] - timestamps[null_model_edge_index[0]]
    non_negative_mask = time_diff > 0
    delta_mask = time_diff <= delta
    time_respecting_mask = non_negative_mask & delta_mask
    edge_index = null_model_edge_index[:, time_respecting_mask]
    
    # identify root nodes with in-degree zero
    in_degree = degree(edge_index[1],num_nodes=g.M)
    roots = torch.where(in_degree==0)[0]

    # create traversable graph
    event_dag = pp.Graph.from_edge_index(edge_index)
    print(event_dag)

    # count all longest time-respecting paths in the temporal graph
    paths = []
    i = 0
    for r in roots:
        print(f'processing root {i+1}/{roots.size(0)}')
        root_paths = routes_from_node(event_dag, r.item(), node_sequence, g.mapping)
        print(f'\t found {len(root_paths)} paths')
        for x in root_paths:
            for p in root_paths[x]:
                paths.append(p)
        i += 1
    print(len(paths))
    return sp_lengths, sp

In [ ]:
sp_lengths, sp = temporal_shortest_paths(t, delta=100)
print(sp_lengths)
print(sp)

In [4]:
t = pp.TemporalGraph.from_csv('ants_1_1.tedges')
print(t)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	t		<class 'torch.Tensor'> -> torch.Size([1911])
	src		<class 'torch.Tensor'> -> torch.Size([1911])
	dst		<class 'torch.Tensor'> -> torch.Size([1911])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'t', 'src', 'dst'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [ ]:
paths = temporal_shortest_paths(t, delta=30)
print(len(paths))

In [5]:
paths = pp.algorithms.time_respecting_paths(t, delta=30)
print(len(paths))

Directed graph with 1910 nodes and 2518 edges

Graph attributes
	num_nodes		<class 'int'>

Processing root 1/597
Processing root 11/597
Processing root 21/597
Processing root 31/597
Processing root 41/597
Processing root 51/597
Processing root 61/597
Processing root 71/597
Processing root 81/597
Processing root 91/597
Processing root 101/597
Processing root 111/597
Processing root 121/597
Processing root 131/597
Processing root 141/597
Processing root 151/597
Processing root 161/597
Processing root 171/597
Processing root 181/597
Processing root 191/597
Processing root 201/597
Processing root 211/597
Processing root 221/597
Processing root 231/597
Processing root 241/597
Processing root 251/597
Processing root 261/597
Processing root 271/597
Processing root 281/597
Processing root 291/597
Processing root 301/597
Processing root 311/597
Processing root 321/597
Processing root 331/597
Processing root 341/597
Processing root 351/597
Processing root 361/597
Processing root 371/597
Processi

In [ ]:
plt.hist(sp_lengths[sp_lengths < 100].flatten().numpy())

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
a = torch.tensor([0,1,2])
b = torch.tensor([1,2,3])
torch.cat([a, b[-1].unsqueeze(dim=0)])

In [ ]:
for x in torch.tensor([0,1]):
    print(x.item())